In [1]:
import json
import time
import h5py
import pandas as pd
from sklearn.datasets import load_files    
from keras.utils import np_utils
import numpy as np
from glob import glob
from PIL import Image
import os
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
import numpy as np

In [3]:
df=pd.read_csv(r"C:\Users\prane\Desktop\2021_2022_2\ML\Assignment 1\CSV_datasetsix_vowel_dataset_with_class.csv")
df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,class
0,255,255,254,255,255,255,252,255,255,255,...,255,255,255,255,255,255,255,255,255,1
1,255,255,254,255,255,254,255,255,255,255,...,255,255,255,255,255,255,255,255,255,1
2,255,254,255,255,255,255,254,255,254,255,...,255,255,255,255,255,255,255,255,255,1
3,254,255,255,255,254,255,255,255,254,255,...,255,255,255,255,255,255,255,255,255,1
4,254,255,255,253,255,255,255,254,255,255,...,255,255,255,255,255,255,255,255,255,1


In [4]:
pixels=[]
for i in range(784):
    pixels.append('pixel'+str(i))

features=pixels
X=df.loc[:,features].values
y=df.loc[:,'class'].values

X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=100)

In [5]:
X_train=X_train/255
X_test=X_test/255

In [6]:
X_train_2D=X_train.reshape(X_train.shape[0],28,28)
X_test_2D=X_test.reshape(X_test.shape[0],28,28)

In [7]:
X_train_2D.shape

(960, 28, 28)

In [8]:
train_tensors=X_train_2D.reshape(X_train_2D.shape[0],28,28,1)
test_tensors=X_test_2D.reshape(X_test_2D.shape[0],28,28,1)
train_labels=y_train
test_labels=y_test

In [9]:
print('Training Tensor', train_tensors.shape)
print('Test set', test_tensors.shape)

Training Tensor (960, 28, 28, 1)
Test set (240, 28, 28, 1)


In [10]:
from keras.preprocessing.image import ImageDataGenerator

data_augmentation=True
batch_size=128
n_epochs=100

In [11]:

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint 

In [12]:
def conv_net_model():
    image_size=28
    model=Sequential()
    model.add(Conv2D(32, (4, 4), activation='relu', padding='same', input_shape=(image_size, image_size, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))   

    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())

    model.add(Dense(512, activation='relu'))              
    model.add(Dropout(0.5))
    
    model.add(Dense(6, activation='softmax'))
    
    model.summary()
    return model

In [13]:
model=conv_net_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        544       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 64)         0

In [14]:
from tensorflow.keras.utils import to_categorical
y_train=y_train-1
train_labels=to_categorical(np.array(y_train))
print(train_labels.shape)
y_test=y_test-1
test_labels=to_categorical(np.array(y_test))

(960, 6)


In [17]:
learning_rate=0.05
decay_rate=learning_rate/n_epochs
momentum=0.8
sgd=SGD(lr=learning_rate,momentum=momentum,decay=decay_rate,nesterov=False)

model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

data_augmentation=False
if data_augmentation:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_tensors)

    # Fit the model on the batches generated by datagen.flow().
    history = model.fit_generator(datagen.flow(train_tensors, train_labels,batch_size=32),
                    steps_per_epoch=int(np.ceil(train_tensors.shape[0] / float(batch_size))),
                    epochs=n_epochs,
                    validation_data = (test_tensors, test_labels), 
                    callbacks = [checkpointer], 
                    verbose = 1)
    
else:
    print('Not using data augmentation.')
    
    #with h5py.File(model_path, 'r') as f:
    history = model.fit(train_tensors, train_labels, epochs = 25, validation_data = (test_tensors, test_labels), batch_size = 128, shuffle = 'batch', verbose=1)
        
#model.save(model_path)
# Final evaluation of the model
scores = model.evaluate(test_tensors, test_labels, verbose=0)

print('Large CNN Error: %.2f%%' % (100-scores[1]*100))
print('Test loss:', scores[0])
print('Test Accuracy : %.2f%%' % scores[1])

#model.save_weights('%d-weights-%f.hdf5' %(timestamp, scores[1]))

Not using data augmentation.
Epoch 1/25
8/8 [==============================] - 1s 88ms/step - loss: 1.7921 - accuracy: 0.1844 - val_loss: 1.7930 - val_accuracy: 0.1417
Epoch 2/25
8/8 [==============================] - 0s 63ms/step - loss: 1.7923 - accuracy: 0.1698 - val_loss: 1.7932 - val_accuracy: 0.1417
Epoch 3/25
8/8 [==============================] - 1s 71ms/step - loss: 1.7922 - accuracy: 0.1719 - val_loss: 1.7924 - val_accuracy: 0.1417
Epoch 4/25
8/8 [==============================] - 1s 70ms/step - loss: 1.7919 - accuracy: 0.1719 - val_loss: 1.7927 - val_accuracy: 0.1417
Epoch 5/25
8/8 [==============================] - 1s 70ms/step - loss: 1.7918 - accuracy: 0.1823 - val_loss: 1.7919 - val_accuracy: 0.1417
Epoch 6/25
8/8 [==============================] - 1s 68ms/step - loss: 1.7920 - accuracy: 0.1583 - val_loss: 1.7923 - val_accuracy: 0.1417
Epoch 7/25
8/8 [==============================] - 1s 67ms/step - loss: 1.7923 - accuracy: 0.1729 - val_loss: 1.7933 - val_accuracy: 0.141

In [64]:
y_train

array([1, 1, 5, 2, 6, 2, 3, 2, 3, 3, 3, 4, 6, 1, 6, 2, 2, 1, 5, 5, 4, 5,
       6, 1, 6, 3, 1, 4, 6, 5, 1, 1, 4, 3, 6, 3, 1, 2, 1, 1, 5, 5, 4, 4,
       3, 3, 1, 4, 5, 2, 2, 2, 5, 4, 6, 5, 6, 1, 4, 5, 4, 1, 2, 2, 3, 2,
       2, 1, 5, 6, 6, 3, 6, 6, 1, 3, 3, 1, 2, 6, 1, 3, 6, 5, 6, 5, 1, 6,
       3, 6, 5, 2, 3, 5, 3, 3, 6, 6, 5, 2, 6, 2, 4, 3, 3, 3, 3, 6, 3, 4,
       5, 6, 3, 3, 1, 5, 4, 2, 2, 6, 6, 3, 4, 6, 2, 2, 6, 2, 1, 6, 1, 5,
       2, 6, 2, 6, 4, 5, 1, 3, 2, 3, 3, 5, 5, 6, 1, 2, 1, 1, 4, 6, 1, 2,
       6, 1, 6, 5, 1, 4, 5, 4, 2, 2, 6, 4, 2, 2, 5, 6, 2, 1, 4, 2, 3, 3,
       2, 3, 3, 6, 5, 2, 5, 5, 4, 4, 3, 1, 1, 1, 3, 2, 4, 5, 4, 1, 4, 2,
       2, 5, 6, 4, 4, 5, 5, 2, 1, 2, 4, 3, 6, 2, 6, 4, 5, 2, 4, 6, 2, 6,
       3, 5, 6, 6, 1, 4, 3, 5, 6, 4, 2, 1, 1, 1, 1, 3, 6, 6, 1, 3, 5, 6,
       4, 2, 3, 4, 6, 5, 2, 4, 2, 3, 1, 2, 5, 2, 6, 5, 2, 4, 5, 3, 5, 6,
       3, 2, 5, 6, 5, 3, 3, 5, 6, 4, 3, 2, 3, 5, 2, 4, 3, 3, 3, 5, 2, 4,
       6, 6, 1, 3, 5, 2, 5, 2, 2, 3, 5, 3, 6, 6, 2,